# Product categorization

Objectif: Regrouper des items en différentes catégories plus générales en se basant sur une analyse sémantique du contenu de la description.
Contrainte: La description ne fait que quelques mots, c'est une entête.
Un exemple d'application pourrait être un site d'échange/ventes entre particulier où le produit est décrit en qq mots par l'utilisateur. Ou alors des messages dont on n'aurait que le sujet, pas le corps du texte.

Le problème s'identifie à du topic modeling mais avec un texte (document) très court

- Exploration / formatage
- Approche Bag of words suivie d'un k-Means
- A pproche Latent Dirichlet Allocation
- Approches hybrides: guided LDA? Autres?

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import nltk

note: le fichier original a été lu en 'latin1' puis ré-encodé en 'utf-8' pour plus de simplicité

In [2]:
data = pd.read_csv('data/data.csv',index_col=0)

In [3]:
data.head(10)

,product_name
0,Appareil à chichis
1,dvd raiponce
2,Raquette de squash artengo sr 840
3,lunette astronomique
4,appareil à chaussons
5,Appareil à fondue au chocolat
6,Petit futé Italie du Nord 2013 - guide
7,Coupe ananas
8,8 flûtes à champagne
9,VTT


In [42]:
import nltk

In [43]:
stemmer = nltk.stem.SnowballStemmer("french")

In [53]:
doc = 'Appareil à chichis'

In [54]:
nltk.word_tokenize(doc)

['Appareil', 'à', 'chichis']

In [55]:
nltk.word_tokenize(doc.lower())

['appareil', 'à', 'chichis']

In [56]:
[stemmer.stem(m) for m in nltk.word_tokenize(doc.lower())]

['appareil', 'à', 'chich']

In [58]:
doc = 'appareil à chaussons'
[stemmer.stem(m) for m in nltk.word_tokenize(doc.lower())]

['appareil', 'à', 'chausson']

In [61]:
doc = 'l\'appareil à chaussons'
nltk.word_tokenize(doc.lower(), language='french')

["l'appareil", 'à', 'chaussons']

In [10]:
stemmer = nltk.stem.SnowballStemmer("french", ignore_stopwords=True)
doc = 'appareil à chaussons'
[stemmer.stem(m) for m in nltk.word_tokenize(doc.lower())]

['appareil', 'à', 'chausson']

In [9]:
stemmer = nltk.stem.SnowballStemmer("french", ignore_stopwords=False)
doc = 'appareil à chaussons'
[stemmer.stem(m) for m in nltk.word_tokenize(doc.lower())]

['appareil', 'à', 'chausson']

In [65]:
from nltk.corpus import stopwords
french_stopwords = set(stopwords.words('french'))
doc = 'appareil à chaussons'
[m for m in nltk.word_tokenize(doc.lower())
     if m.lower() not in french_stopwords]

['appareil', 'chaussons']

In [4]:
test_corpus = data.head(10)


In [5]:
test_corpus

,product_name
0,Appareil à chichis
1,dvd raiponce
2,Raquette de squash artengo sr 840
3,lunette astronomique
4,appareil à chaussons
5,Appareil à fondue au chocolat
6,Petit futé Italie du Nord 2013 - guide
7,Coupe ananas
8,8 flûtes à champagne
9,VTT


In [8]:
list(test_corpus['product_name'])

['Appareil à chichis',
 'dvd raiponce',
 'Raquette de squash artengo sr 840',
 'lunette astronomique',
 'appareil à chaussons',
 'Appareil à fondue au chocolat',
 'Petit futé Italie du Nord 2013 - guide',
 'Coupe ananas',
 '8 flûtes à champagne',
 'VTT']

In [12]:
def stemming_corpus(corpus):
    ## stemming corpus
    # le stemmer doit être appliqué sur des tokens
    # tous les mots des documents sont processés 
    # mais je conserve au final le format de corpus
    from nltk.corpus import stopwords
    french_stopwords = set(stopwords.words('french'))
    tokenizer = nltk.RegexpTokenizer(r'\b\w\w+\b')
    stemmer = stemmer = nltk.stem.SnowballStemmer("french")
    stemmed_corpus = []
    for doc in corpus:
        s = ' '
        stemmed_corpus.extend(
            [s.join([stemmer.stem(w) for w in tokenizer.tokenize(doc) 
                     if w.lower() not in french_stopwords])]
        )
    return(stemmed_corpus)


In [13]:
stemming_corpus(list(test_corpus['product_name']))

['appareil chich',
 'dvd raiponc',
 'raquet squash artengo sr 840',
 'lunet astronom',
 'appareil chausson',
 'appareil fondu chocolat',
 'pet fut ital nord 2013 guid',
 'coup anan',
 'flût champagn',
 'vtt']

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(min_df=0, max_df=0.95, # ignore termes présents ds plus 95% doc
                             ngram_range=(1, 2)) # si je veux considérer ts les unigrammes et tous les bigrammes
corpus = stemming_corpus(list(test_corpus['product_name']))
tfidf_vect.fit(corpus)

vocabulaire = tfidf_vect.vocabulary_
print('Vocabulary size: {}'.format(len(tfidf_vect.vocabulary_)))
print('10 first features:\n{}'.format(tfidf_vect.get_feature_names()[:10]))


Vocabulary size: 42
10 first features:
['2013', '2013 guid', '840', 'anan', 'appareil', 'appareil chausson', 'appareil chich', 'appareil fondu', 'artengo', 'artengo sr']


In [17]:
X_train = tfidf_vect.transform(corpus)

In [18]:
X_train

<10x42 sparse matrix of type '<class 'numpy.float64'>'
	with 44 stored elements in Compressed Sparse Row format>

In [19]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(X_train)

In [20]:
kmeans.labels_

array([1, 0, 0, 0, 1, 1, 0, 0, 0, 0])

In [21]:
test_corpus['labels'] = kmeans.labels_
test_corpus

C:\Users\Nicolas\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,product_name,labels
0,Appareil à chichis,1
1,dvd raiponce,0
2,Raquette de squash artengo sr 840,0
3,lunette astronomique,0
4,appareil à chaussons,1
5,Appareil à fondue au chocolat,1
6,Petit futé Italie du Nord 2013 - guide,0
7,Coupe ananas,0
8,8 flûtes à champagne,0
9,VTT,0


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(min_df=5, max_df=0.95, # ignore termes présents ds plus 95% doc
                             ngram_range=(1, 2)) # si je veux considérer ts les unigrammes et tous les bigrammes
corpus = stemming_corpus(list(data['product_name']))
tfidf_vect.fit(corpus)

vocabulaire = tfidf_vect.vocabulary_
print('Vocabulary size: {}'.format(len(tfidf_vect.vocabulary_)))
print('10 first features:\n{}'.format(tfidf_vect.get_feature_names()[:10]))


Vocabulary size: 592
10 first features:
['1200', '14', '14 4v', '3d', '400w', '44', '4v', '6000', '6000 pro', '90']


In [23]:
X_train = tfidf_vect.transform(corpus)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, random_state=0).fit(X_train)

In [32]:
data.iloc[np.where(kmeans.labels_==0)].head(10)

,product_name
215,Scie sauteuse
227,Scie sauteuse
287,Scie sauteuse 350W
334,Scie Sauteuse
353,Scie sauteuse
378,Scie sauteuse
412,Scie sauteuse
486,Scie sauteuse Bosch PST 700 PE
523,Scie sauteuse
584,Scie sauteuse


In [33]:
data.iloc[np.where(kmeans.labels_==1)].head(10)

,product_name
0,Appareil à chichis
4,appareil à chaussons
16,Appareil à chichis
30,Appareil à raclette
31,appareil à raclette
45,Appareil a raclette 8 personnes
46,Appareil à raclette
74,Appareil à raclette
76,Appareil a raclette
97,Appareil à chichis


In [34]:
data.iloc[np.where(kmeans.labels_==2)].head(10)

,product_name
147,Ponceuse BOSCH
284,Ponceuse vibrante ROTOTECH
455,Ponceuse BOSCH
478,Ponceuse Bosch PSM 800 A
479,PONCEUSE TRIANGULAIRE
480,Ponceuse électrique
549,Ponceuse électrique
550,Ponceuse Bosh
583,Ponceuse électrique
687,Ponceuse BOSCH


In [35]:
data.iloc[np.where(kmeans.labels_==3)].head(10)

,product_name
141,Diable
245,Diable
352,Diable
361,Diable
411,Diable pliant
436,Diable
501,Diable
728,Diable pliable
738,Diable 3 roues
843,diable


In [36]:
data.iloc[np.where(kmeans.labels_==4)].head(10)

,product_name
1,dvd raiponce
2,Raquette de squash artengo sr 840
3,lunette astronomique
5,Appareil à fondue au chocolat
6,Petit futé Italie du Nord 2013 - guide
7,Coupe ananas
8,8 flûtes à champagne
9,VTT
10,Playstation 3
11,Mini laser


In [45]:
corpus[1:5]

['dvd raiponc',
 'raquet squash artengo sr 840',
 'lunet astronom',
 'appareil chausson']

In [51]:
np.where(kmeans.labels_==3)

(array([ 141,  245,  352,  361,  411,  436,  501,  728,  738,  843,  844,
         845,  846,  895,  896,  897,  988, 1069, 1120, 1121, 1122, 1181,
        1245, 1306, 1307, 1410, 1557, 1560, 1669, 1796, 1797, 1822, 1833,
        1837, 1839, 1840, 1841, 1842, 1844, 1973, 1986, 1987, 2018, 2023,
        2040, 2053, 2092, 2095, 2100, 2112, 2118, 2121, 2157, 2161, 2163,
        2173, 2174, 2177, 2315, 2320, 2321, 2341, 2453, 2519, 2563, 2678,
        2679, 2682, 2683, 2684, 2705, 2706, 2707, 2795, 2845, 2846, 2849,
        2870, 2871, 2885, 3104, 3105, 3106, 3137, 3211, 3262, 3279, 3296,
        3334, 3346, 3373, 3451, 3464, 3465, 3471, 3489, 3490, 3491, 3540,
        3621, 3636, 3637, 3661, 3672, 3673, 3674, 3818, 3819, 3853, 3894,
        3965, 3966, 3989, 3992, 3993, 3996, 3998, 3999, 4000, 4060, 4066,
        4067, 4128, 4341, 4391, 4392, 4430, 4465, 4485, 4486, 4487, 4488,
        4489, 4491, 4492, 4493, 4494, 4495, 4497, 4498, 4499, 4503, 4548,
        4618, 4620, 4656, 4658, 4682, 

In [52]:
corpus[kmeans.labels_==3]

TypeError: only integer scalar arrays can be converted to a scalar index

In [54]:
np.array([1,5])

array([1, 5])

In [55]:
tfidf_vect.inverse_transform(corpus[:10])

[array(['1200', '14', '14 4v', '3d', '400w', '44', '4v', '6000',
        '6000 pro', '90'], dtype='<U19')]

In [56]:
X_train

<5206x592 sparse matrix of type '<class 'numpy.float64'>'
	with 13651 stored elements in Compressed Sparse Row format>

In [57]:
X_train[np.where(kmeans.labels_==3)]

<164x592 sparse matrix of type '<class 'numpy.float64'>'
	with 171 stored elements in Compressed Sparse Row format>

In [58]:
tfidf_vect.inverse_transform(X_train[np.where(kmeans.labels_==3)])

[array(['diabl'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl', 'pli'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl', 'pliabl'], dtype='<U19'),
 array(['diabl', 'rou'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl', 'pli'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl', 'déménag'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 array(['diabl'], dtype='<U19'),
 

In [69]:
tfidf_vect.inverse_transform(X_train[np.where(kmeans.labels_==3)])[4][0:]

array(['diabl', 'pli'], dtype='<U19')

In [73]:
[w[0] for w in tfidf_vect.inverse_transform(X_train[np.where(kmeans.labels_==3)])]

['diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',
 'diabl',


In [74]:
[w for w in tfidf_vect.inverse_transform(X_train[np.where(kmeans.labels_==3)])[4]]

['diabl', 'pli']